In [1]:
# Tập hợp các tài liệu mẫu
corpus = {
    1: "Exploring the vibrant world of tropical fish in a home aquarium.",
    2: "The challenges of maintaining a healthy aquarium for your fish.",
    3: "A guide to the most popular aquarium fish species.",
    4: "Tropical birds are known for their bright colors.",
    5: "Many people enjoy the hobby of keeping fish.",
    6: "What is the difference between tropical fish and cold-water fish?"
}

## Search structured query

In [2]:
def search_structured_query(corpus):
    """
    Thực hiện một truy vấn có cấu trúc để tìm các tài liệu phù hợp.
    Query: #combine(#od:1(tropical fish), #od:1(aquarium fish), fish)
    """
    print("--- Bắt đầu thực hiện Truy vấn có cấu trúc ---")
    matching_docs = []
    
    for doc_id, content in corpus.items():
        # Chuẩn hóa văn bản về chữ thường để tìm kiếm không phân biệt chữ hoa/thường
        content_lower = content.lower()
        
        # Kiểm tra các điều kiện của truy vấn
        # 1. Chứa cụm từ "tropical fish"
        # 2. Hoặc chứa cụm từ "aquarium fish"
        # 3. Hoặc chứa từ "fish"
        if "tropical fish" in content_lower or \
           "aquarium fish" in content_lower or \
           " fish" in content_lower or \
           content_lower.startswith("fish"): # Để bắt trường hợp "fish" ở đầu câu
            
            matching_docs.append(doc_id)
            print(f"  [+] Tìm thấy kết quả trong Tài liệu {doc_id}: '{content}'")
            
    print(f"\n---> Tổng số tài liệu phù hợp: {len(matching_docs)}")
    return matching_docs

# Chạy demo
structured_results = search_structured_query(corpus)

--- Bắt đầu thực hiện Truy vấn có cấu trúc ---
  [+] Tìm thấy kết quả trong Tài liệu 1: 'Exploring the vibrant world of tropical fish in a home aquarium.'
  [+] Tìm thấy kết quả trong Tài liệu 2: 'The challenges of maintaining a healthy aquarium for your fish.'
  [+] Tìm thấy kết quả trong Tài liệu 3: 'A guide to the most popular aquarium fish species.'
  [+] Tìm thấy kết quả trong Tài liệu 5: 'Many people enjoy the hobby of keeping fish.'
  [+] Tìm thấy kết quả trong Tài liệu 6: 'What is the difference between tropical fish and cold-water fish?'

---> Tổng số tài liệu phù hợp: 5


## Search document distributed

In [3]:
# Chia corpus thành 2 "mảnh" (shards), mô phỏng 2 máy chủ khác nhau
shard1 = {k: v for k, v in corpus.items() if k <= 3}
shard2 = {k: v for k, v in corpus.items() if k > 3}

def search_document_distributed(shards):
    """
    Mô phỏng tìm kiếm phân tán theo tài liệu.
    Truy vấn được gửi đến tất cả các shard và kết quả được tổng hợp lại.
    """
    print("\n--- Bắt đầu thực hiện Đánh giá Phân tán theo Tài liệu ---")
    all_results = []
    
    # Gửi truy vấn đến từng shard
    print("Gửi truy vấn đến Shard 1...")
    results1 = search_structured_query(shard1)
    all_results.extend(results1)
    
    print("\nGửi truy vấn đến Shard 2...")
    results2 = search_structured_query(shard2)
    all_results.extend(results2)
    
    # Loại bỏ các kết quả trùng lặp (nếu có) và sắp xếp
    final_results = sorted(list(set(all_results)))
    
    print(f"\n---> Kết quả cuối cùng sau khi tổng hợp: {final_results}")
    return final_results

# Chạy demo
distributed_results = search_document_distributed([shard1, shard2])


--- Bắt đầu thực hiện Đánh giá Phân tán theo Tài liệu ---
Gửi truy vấn đến Shard 1...
--- Bắt đầu thực hiện Truy vấn có cấu trúc ---
  [+] Tìm thấy kết quả trong Tài liệu 1: 'Exploring the vibrant world of tropical fish in a home aquarium.'
  [+] Tìm thấy kết quả trong Tài liệu 2: 'The challenges of maintaining a healthy aquarium for your fish.'
  [+] Tìm thấy kết quả trong Tài liệu 3: 'A guide to the most popular aquarium fish species.'

---> Tổng số tài liệu phù hợp: 3

Gửi truy vấn đến Shard 2...
--- Bắt đầu thực hiện Truy vấn có cấu trúc ---
  [+] Tìm thấy kết quả trong Tài liệu 5: 'Many people enjoy the hobby of keeping fish.'
  [+] Tìm thấy kết quả trong Tài liệu 6: 'What is the difference between tropical fish and cold-water fish?'

---> Tổng số tài liệu phù hợp: 2

---> Kết quả cuối cùng sau khi tổng hợp: [1, 2, 3, 5, 6]


## Inverted Index phân tán

In [4]:
# Mô phỏng một Inverted Index phân tán
# Server 1 quản lý các từ bắt đầu bằng a-m
# Server 2 quản lý các từ bắt đầu bằng n-z
distributed_inverted_index = {
    'server1': {
        'aquarium': [1, 2, 3],
        'fish': [1, 2, 3, 5, 6],
        'guide': [3],
        'healthy': [2]
    },
    'server2': {
        'tropical': [1, 4, 6],
        'world': [1],
        'species': [3]
    }
}

def search_term_distributed(query_terms, index):
    """
    Mô phỏng tìm kiếm phân tán theo từ.
    Hệ thống sẽ hỏi các server khác nhau để lấy danh sách tài liệu cho từng từ.
    """
    print("\n--- Bắt đầu thực hiện Đánh giá Phân tán theo Từ ---")
    print(f"Truy vấn: Tìm tài liệu chứa '{' và '.join(query_terms)}'")
    
    # Lấy danh sách tài liệu (postings) từ các server tương ứng
    postings = []
    for term in query_terms:
        if 'a' <= term[0] <= 'm':
            server = 'server1'
        else:
            server = 'server2'
        
        term_postings = index[server].get(term, [])
        postings.append(set(term_postings))
        print(f"  - Từ '{term}' được tìm thấy trên '{server}'. Tài liệu chứa nó: {term_postings}")
        
    # Giao (AND) các danh sách để tìm tài liệu chứa TẤT CẢ các từ
    if not postings:
        return []
    
    result_set = postings[0].intersection(*postings[1:])
    
    print(f"\n---> Kết quả cuối cùng (giao của các danh sách): {sorted(list(result_set))}")
    return sorted(list(result_set))

# Chạy demo cho truy vấn "tropical fish"
term_results = search_term_distributed(['tropical', 'fish'], distributed_inverted_index)


--- Bắt đầu thực hiện Đánh giá Phân tán theo Từ ---
Truy vấn: Tìm tài liệu chứa 'tropical và fish'
  - Từ 'tropical' được tìm thấy trên 'server2'. Tài liệu chứa nó: [1, 4, 6]
  - Từ 'fish' được tìm thấy trên 'server1'. Tài liệu chứa nó: [1, 2, 3, 5, 6]

---> Kết quả cuối cùng (giao của các danh sách): [1, 6]


## Caching

In [5]:
class QueryProcessor:
    def __init__(self, corpus):
        self.corpus = corpus
        self.cache = {} # Bộ nhớ cache để lưu kết quả

    def search(self, query):
        """
        Thực hiện tìm kiếm có sử dụng cache.
        """
        print(f"\n--- Thực hiện truy vấn: '{query}' ---")
        
        # 1. Kiểm tra xem truy vấn đã có trong cache chưa
        if query in self.cache:
            print("  [CACHE HIT] Lấy kết quả từ cache.")
            return self.cache[query]
        
        # 2. Nếu không có, thực hiện tìm kiếm (giả sử đây là một quá trình tốn kém)
        print("  [CACHE MISS] Không tìm thấy trong cache. Đang thực hiện tính toán...")
        results = []
        for doc_id, content in self.corpus.items():
            if query in content.lower():
                results.append(doc_id)
        
        # 3. Lưu kết quả vào cache
        print(f"  Lưu kết quả '{results}' vào cache cho truy vấn '{query}'.")
        self.cache[query] = results
        
        return results

# Chạy demo
processor = QueryProcessor(corpus)

# Lần 1: Truy vấn "aquarium"
results1 = processor.search("aquarium")
print(f"---> Kết quả: {results1}")

# Lần 2: Truy vấn lại "aquarium"
results2 = processor.search("aquarium")
print(f"---> Kết quả: {results2}")

# Lần 3: Truy vấn mới "colors"
results3 = processor.search("colors")
print(f"---> Kết quả: {results3}")


--- Thực hiện truy vấn: 'aquarium' ---
  [CACHE MISS] Không tìm thấy trong cache. Đang thực hiện tính toán...
  Lưu kết quả '[1, 2, 3]' vào cache cho truy vấn 'aquarium'.
---> Kết quả: [1, 2, 3]

--- Thực hiện truy vấn: 'aquarium' ---
  [CACHE HIT] Lấy kết quả từ cache.
---> Kết quả: [1, 2, 3]

--- Thực hiện truy vấn: 'colors' ---
  [CACHE MISS] Không tìm thấy trong cache. Đang thực hiện tính toán...
  Lưu kết quả '[4]' vào cache cho truy vấn 'colors'.
---> Kết quả: [4]
